In [1]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6893.66it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3323.25it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 2964446.21it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2118/2118 [00:00<00:00, 4878.11it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 18841/18841 [00:07<00:00, 2572.20it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 122/122 [00:00<00:00, 4624.45it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1115/1115 [00:00<00:00, 2609.60it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 237/237 [00:00<00:00, 5193.33it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2300/2300 [00:00<00:00, 2579.14it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DSSM.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          2893500     text_left[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 300)          2893500     text_right[0][0]                 
_______________________________________________________________________________

In [7]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [8]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=1, num_neg=4, batch_size=64, shuffle=True)
len(train_generator)

16

In [9]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/20
16/16 [==============================] - 3s 175ms/step - loss: 1.5564
Validation: normalized_discounted_cumulative_gain@3(0): 0.45901904458281206 - normalized_discounted_cumulative_gain@5(0): 0.5475429093192175 - mean_average_precision(0): 0.4911167747235369
Epoch 2/20
16/16 [==============================] - 2s 96ms/step - loss: 1.2718
Validation: normalized_discounted_cumulative_gain@3(0): 0.4654009025837788 - normalized_discounted_cumulative_gain@5(0): 0.5392725183775516 - mean_average_precision(0): 0.48941174032387735
Epoch 3/20
16/16 [==============================] - 2s 100ms/step - loss: 1.1539
Validation: normalized_discounted_cumulative_gain@3(0): 0.4791185363644891 - normalized_discounted_cumulative_gain@5(0): 0.5538418263100713 - mean_average_precision(0): 0.5071942064030672
Epoch 4/20
16/16 [==============================] - 2s 99ms/step - loss: 1.0995
Validation: normalized_discounted_cumulative_gain@3(0): 0.4886745351048811 - normalized_discounted_cumulative_ga